### Importing libraries

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql import types

#### Configuration variables

In [2]:
credentials_location = "mage-oregon-crime/oregon-crime-e130b790c75d.json"

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

#### Creating the context with the credentials

In [3]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

your 131072x1 screen size is bogus. expect trouble
24/04/12 11:38:11 WARN Utils: Your hostname, dashel resolves to a loopback address: 127.0.1.1; using 172.31.179.36 instead (on interface eth0)
24/04/12 11:38:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/12 11:38:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


#### Creating Spark session

In [6]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

#### Schema for the oregon crime data. 

In [4]:
schema = types.StructType([
    types.StructField('_c0', types.StringType(), True), 
    types.StructField('Address', types.StringType(), True), 
    types.StructField('CaseNumber', types.StringType(), True), 
    types.StructField('CrimeAgainst', types.StringType(), True), 
    types.StructField('Neighborhood', types.StringType(), True), 
    types.StructField('OccurDate', types.DateType(), True), 
    types.StructField('OccurTime', types.TimestampType(), True), 
    types.StructField('OffenseCategory', types.StringType(), True), 
    types.StructField('OffenseType', types.StringType(), True), 
    types.StructField('OpenDataLat', types.StringType(), True), 
    types.StructField('OpenDataLon', types.StringType(), True), 
    types.StructField('ReportDate', types.DateType(), True), 
    types.StructField('OffenseCount', types.IntegerType(), True)
    ])

#### Loading data for the Uniform Crime Stats

In [7]:
df_uni_arrests = spark.read.parquet('gs://oregon-crime-bucket/uniform_data/arrests_to_2024.parquet')
df_uni_leoka = spark.read.parquet('gs://oregon-crime-bucket/uniform_data/leoka_to_2024.parquet')
df_uni_offenses = spark.read.parquet('gs://oregon-crime-bucket/uniform_data/offences_to_2024.parquet')
df_uni_victims = spark.read.parquet('gs://oregon-crime-bucket/uniform_data/victims_to_2024.parquet')

#### Loading Oregon Crime data

In [8]:
df_crime_data = spark.read.option('header', 'true').schema(schema).csv('gs://oregon-crime-bucket/crime_data_csv/oregon_crime_2015_2023.csv')

#### Dropping first column that are Indexees comming from the csv file

In [9]:
df_crime_data = df_crime_data.drop('_c0')

In [15]:
df_crime_data.show(5)

+-------+-----------+------------+---------------+----------+---------+--------------------+--------------------+-----------+-----------+----------+------------+
|Address| CaseNumber|CrimeAgainst|   Neighborhood| OccurDate|OccurTime|     OffenseCategory|         OffenseType|OpenDataLat|OpenDataLon|ReportDate|OffenseCount|
+-------+-----------+------------+---------------+----------+---------+--------------------+--------------------+-----------+-----------+----------+------------+
|   NULL| 15-X197430|      Person|       Piedmont|2015-05-12| 14:00:00|    Assault Offenses|        Intimidation|       NULL|       NULL|2015-05-12|           1|
|   NULL|15-X4282999|      Person|   Buckman West|2015-05-01| 21:43:00|    Assault Offenses|      Simple Assault|       NULL|       NULL|2015-05-01|           1|
|   NULL|15-X4283033|      Person|University Park|2015-05-01| 16:25:00|    Assault Offenses|      Simple Assault|       NULL|       NULL|2015-05-01|           1|
|   NULL|15-X4283218|      P

#### Keeping only the time in the "OccurTime" column

In [11]:
df_crime_data = df_crime_data.withColumn("OccurTime", F.date_format("OccurTime", "HH:mm:ss"))

### Cheking Data sets

In [23]:
print(df_uni_offenses.printSchema())
print(df_uni_leoka.printSchema())
print(df_uni_arrests.printSchema())
print(df_uni_victims.printSchema())

root
 |-- Agency Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- IncidentDate: date (nullable = true)
 |-- NIBRS Crime Against: string (nullable = true)
 |-- NIBRS Crime Description: string (nullable = true)
 |-- NIBRS Report Title: string (nullable = true)
 |-- Distinct Offenses: long (nullable = true)

None
root
 |-- Agency Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- IncidentDate: date (nullable = true)
 |-- LEOKA Assignment Type (NIBRS): string (nullable = true)
 |-- LEOKA Circumstance: string (nullable = true)
 |-- NIBRS Crime Description: string (nullable = true)
 |-- NIBRS Report Title: string (nullable = true)
 |-- Victim Age (group): string (nullable = true)
 |-- Victim Gender: string (nullable = true)
 |-- Victim Race: string (nullable = true)
 |-- Distinct Offense Victims: long (nullable = true)

None
root
 |-- Agency Name: string (nullable = true)
 |-- Arrestee Age (group): string (nullable = true)
 |-- Arrestee Gender:

In [14]:
df_crime_data.show(5)

+-------+-----------+------------+---------------+----------+---------+--------------------+--------------------+-----------+-----------+----------+------------+
|Address| CaseNumber|CrimeAgainst|   Neighborhood| OccurDate|OccurTime|     OffenseCategory|         OffenseType|OpenDataLat|OpenDataLon|ReportDate|OffenseCount|
+-------+-----------+------------+---------------+----------+---------+--------------------+--------------------+-----------+-----------+----------+------------+
|   NULL| 15-X197430|      Person|       Piedmont|2015-05-12| 14:00:00|    Assault Offenses|        Intimidation|       NULL|       NULL|2015-05-12|           1|
|   NULL|15-X4282999|      Person|   Buckman West|2015-05-01| 21:43:00|    Assault Offenses|      Simple Assault|       NULL|       NULL|2015-05-01|           1|
|   NULL|15-X4283033|      Person|University Park|2015-05-01| 16:25:00|    Assault Offenses|      Simple Assault|       NULL|       NULL|2015-05-01|           1|
|   NULL|15-X4283218|      P

In [17]:
df_uni_arrests.show(5)

+----------------+--------------------+---------------+--------------------+----------+----------+-------------------+-----------------------+------------------+------------------------+
|     Agency Name|Arrestee Age (group)|Arrestee Gender|       Arrestee Race|    County|      Date|NIBRS Crime Against|NIBRS Crime Description|NIBRS Report Title|Distinct Arrest Offenses|
+----------------+--------------------+---------------+--------------------+----------+----------+-------------------+-----------------------+------------------+------------------------+
|Beaverton PD MIP|                0-10|           Male|Black or African ...|Washington|2021-10-14|             Person|         Simple Assault|    Simple Assault|                       1|
|     Bend PD MIP|                0-10|         Female|             Unknown| Deschutes|2020-08-04|              Other|   Oregon Specific C...|            Part 3|                       1|
|     Bend PD MIP|                0-10|         Female|          

In [19]:
df_uni_leoka.show(5)

+--------------------+---------+------------+-----------------------------+--------------------+-----------------------+------------------+------------------+-------------+-----------+------------------------+
|         Agency Name|   County|IncidentDate|LEOKA Assignment Type (NIBRS)|  LEOKA Circumstance|NIBRS Crime Description|NIBRS Report Title|Victim Age (group)|Victim Gender|Victim Race|Distinct Offense Victims|
+--------------------+---------+------------+-----------------------------+--------------------+-----------------------+------------------+------------------+-------------+-----------+------------------------+
|        Clackamas SO|Clackamas|  2023-06-10|         One Man Vehicle -...|Robberies in Prog...|         Simple Assault|    Simple Assault|                20|       Female|      White|                       1|
|      Gresham PD MIP|Multnomah|  2021-10-24|             Other - Assisted|           All Other|     Aggravated Assault|Aggravated Assault|                20|  

In [24]:
df_uni_victims.show(5)

+-----------+------+------------+-------------------+-----------------------+------------------+------------------+-------------+-----------+----------+------------------------+
|Agency Name|County|IncidentDate|NIBRS Crime Against|NIBRS Crime Description|NIBRS Report Title|Victim Age (group)|Victim Gender|Victim Race|VictimType|Distinct Offense Victims|
+-----------+------+------------+-------------------+-----------------------+------------------+------------------+-------------+-----------+----------+------------------------+
|  Marion SO|Marion|  2020-01-02|              Other|        OUCR Only Crime|            Part 3|              NULL|         NULL|       NULL|      NULL|                       0|
|  Marion SO|Marion|  2020-01-03|              Other|        OUCR Only Crime|            Part 3|              NULL|         NULL|       NULL|      NULL|                       0|
|  Marion SO|Marion|  2020-01-08|              Other|        OUCR Only Crime|            Part 3|              